In [74]:
import numpy as np
import gym
from IPython.display import clear_output
import time
import tensorflow as tf

In [89]:
Map= 'SFFFHFFFFFFFFFFG'
env_name= "FrozenLake-v0"
amap=np.asarray(Map, dtype='c')
shape=int(np.sqrt(amap.shape[0]))
amap=amap.reshape(shape,shape)
env=gym.make(env_name, desc=amap).unwrapped

In [125]:
class FrozenLake():
    
    def __init__(self, env, discount_rate= 1.0, learning_rate= 0.25):     
        self.state_space  = env.observation_space.n
        self.action_space = env.action_space.n
        self.discount_rate= discount_rate
        self.learning_rate= learning_rate
        self.eps= 1.0
        self.build_model()
        
        self.sess= tf.Session()
        self.sess.run(tf.global_variables_initializer())
    
    def build_model (self):
        tf.reset_default_graph()
        
        self.state= tf.placeholder(tf.int32, shape= [1])
        self.action= tf.placeholder(tf.int32, shape= [1])
        self.target= tf.placeholder(tf.float32, shape= [1])
        
        self.state_b= tf.one_hot(self.state, depth=self.state_space)
        self.action_b= tf.one_hot(self.action, depth=self.action_space)
        
        self.q_state_1= tf.layers.dense(self.state_b, units=100, name="q_table_1")
        #self.q_state_2= tf.layers.dense(self.q_state_1, units=50, name="q_table_2")
        self.q_state_3= tf.layers.dense(self.q_state_1, units=self.action_space, name="q_table_3")
        self.q_value= tf.reduce_sum(tf.multiply(self.q_state_3, self.action_b), axis=1)
        
        self.loss= tf.reduce_sum(tf.square(self.target - self.q_value))
        self.optimizer= tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        
    def get_action(self, cur_state):
        q_value= self.sess.run(self.q_state_3, feed_dict= {self.state: [cur_state]})
        
        greedy= np.argmax(q_value)
        random= np.random.randint(self.action_space)

        if np.random.random() < self.eps:
            return(random)
        else:
            return(greedy)
        
    def get_action_test(self, cur_state):
        q_value= self.sess.run(self.q_state_3, feed_dict= {self.state: [cur_state]})
        
        return(np.argmax(q_value))
    
    def train (self, experience):
        cur_state, action, next_state, reward, done= experience
        
        if experience[4]:
            target=0
            self.eps= self.eps*0.99
        else:
            q_next= self.sess.run(self.q_state_3, feed_dict={self.state: [next_state]})
            target = reward + self.discount_rate*np.max(q_next)
            
        self.sess.run(self.optimizer, feed_dict= {self.state: [cur_state], self.action: [action], self.target: [target]})
        
    def __del__(self):
        self.sess.close()
        

In [126]:
agent= FrozenLake(env)

In [129]:
Total_reward=0

for i in range(100):
    cur_state= env.reset()
    
    done= False
    while not done:
        action = agent.get_action(cur_state)
        next_state, reward, done, info= env.step(action)

        agent.train((cur_state, action, next_state, reward, done))
        Total_reward+=reward
        
        cur_state= next_state
    
        print(agent.eps)
        '''
        with tf.variable_scope("q_table",reuse= True):
            weights= agent.sess.run(tf.get_variable("kernel"))
            print(weights)
        '''

        env.render()
        print(Total_reward)
        #time.sleep(0.3)
        clear_output(wait=True)

0.04904089407128576
  (Up)
SFFF
HFFF
FFFF
FFFG
35.0
